A minimal approach to use transformers using the simpletransformers package (https://github.com/ThilinaRajapakse/simpletransformers/) that allows for:
1. Easy switching between different pre-trained models
2. Simple hyperparameter tuning
3. Basic k-fold cross validation
4. Zero pre-processing of data

However, language model fine-tuning is not available with this method.

Updates:
12 Jan 2020 - Update to set random seed to ensure deterministic prediction.

In [1]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 264 kB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 8.0 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 124 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 8.2 MB/s eta 0:00:011
     |████████████████████████████████| 17.6 MB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 170 kB 5.

     |████████████████████████████████| 63 kB 1.1 MB/s eta 0:00:01


  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16171 sha256=8395815002afca2acd02b5cb0c1f765efac61ebab66e48cc0f5b58bff92b89bc
  Stored in directory: /Users/jc/Library/Caches/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13451 sha256=1d4dc39417ca0b59fd817d361a6fad384f5e407fcb007b97c916db1d36632055
  Stored in directory: /Users/jc/Library/Caches/pip/wheels/b7/a5/68/fe632054a5eadd531c7a49d740c50eb6adfbeca822b4eab8d4
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6488 sha256=2cbc9be79a6837677011f50261f900a40990713a34aa98f48c004778ad482990
  Stored in directory: /Users/jc/Library/Caches/pip/wheels/9f/69/d1/50b39b308a87998eaf5c1d9095e5a5bd2ad98501e2b7936d36
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21493 sha256=b425f5e5a01633c29b0806212c156b11e17f2c3d87c499af1b643a93d60f2b40
  Stored in directory: /Users/

In [3]:
import os, re, string
import random

import numpy as np
import pandas as pd
import sklearn

import torch

from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [4]:
seed = 1337

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [6]:
# Zero pre-processing of training data
# I have tried doing some preprcessing/cleaning but the result 
# does not seem significant
train_data = pd.read_csv('train.csv')
train_data = train_data[['text', 'target']]

# Check default value of model and tweak

In [9]:
# Using 'bert-large-uncased' here. For a list of other models, please refer to 
# https://github.com/ThilinaRajapakse/simpletransformers/#current-pretrained-models 
bert_uncased = ClassificationModel('bert', 'bert-large-uncased',use_cuda= False) 

# Print out all the default arguments for reference
bert_uncased.args

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

ClassificationArgs(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=True, adafactor_scale_parameter=True, adafactor_warmup_init=True, adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=False, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, manual_seed=None, max_grad_norm=1.0, max_seq_length=128, model_nam

In [10]:
# This is where we can tweak based on the default arguments above
custom_args = {'fp16': False, # not using mixed precision 
               'train_batch_size': 4, # default is 8
               'gradient_accumulation_steps': 2,
               'do_lower_case': True,
               'learning_rate': 1e-05, # using lower learning rate
               'overwrite_output_dir': True, # important for CV
               'num_train_epochs': 2} # default is 1

# 5-Fold CV

In [13]:
n=5
kf = KFold(n_splits=n, random_state=seed, shuffle=True)
results = []

for train_index, val_index in kf.split(train_data):
    train_df = train_data.iloc[train_index]
    val_df = train_data.iloc[val_index]
    
    model = ClassificationModel('bert', 'bert-base-uncased', args=custom_args,use_cuda = False) 
    model.train_model(train_df)
    result, model_outputs, wrong_predictions = model.eval_model(val_df, acc=sklearn.metrics.accuracy_score)
    print(result['acc'])
    results.append(result['acc'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

/Users/jc/opt/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:615: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


KeyboardInterrupt: 

In [12]:
for i, result in enumerate(results, 1):
    print(f"Fold-{i}: {result}")
    
print(f"{n}-fold CV accuracy result: Mean: {np.mean(results)} Standard deviation:{np.std(results)}")

5-fold CV accuracy result: Mean: nan Standard deviation:nan


/Users/jc/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jc/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/jc/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/jc/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/Users/jc/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# Full Training

In [ ]:
model = ClassificationModel('bert', 'bert-base-uncased', args=custom_args) 
model.train_model(train_data)

# Predict

In [ ]:
test_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
predictions, raw_outputs = model.predict(test_data['text'])

sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = predictions
sample_submission.to_csv("submission.csv", index=False)